In [2]:
import datetime as dt
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine 
from sqlalchemy import text
import geoalchemy2
from shapely import wkt
from pandarallel import pandarallel
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import shapely
#activate high interaction shell so print() is not necessary to show output. https://stackoverflow.com/questions/31764006/ipython-notebook-display-every-line-output-without-print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

TEST_THE_MODEL = 'binaryModel'
#TEST_THE_MODEL = 'multiclassModel'
#TEST_THE_MODEL = 'speedFilter'

if TEST_THE_MODEL == 'multiclassModel':
    print('Testing the model ' + TEST_THE_MODEL)
    #path = 'results/multiclassModel/'
    labelsDict = {0:'A', 1:'C', 2:'N'}
    MLmodel = joblib.load('results/multiclassModel/fittedModel_multiClassModel.pkl')

if TEST_THE_MODEL == 'binaryModel':
    print('Testing the model ' + TEST_THE_MODEL)
    #path = 'results/binaryModel/'
    labelsDict = {0:'AN', 1:'C'}
    MLmodel = joblib.load('results/binaryModel/fittedModel_binaryModel.pkl')

if TEST_THE_MODEL == 'speedFilter':
    print('Testing the model ' + TEST_THE_MODEL)
    #pathImport = 'results/speedFilter/'
    #pathExport = '/home/joan/Desktop/ICM_SAP/papers/paperPeixBlau/dataAnalysis/ModelFishingEffortCheck/speedFilter/'
    labelsDict1 = {False:'AN', True:'C'}

#create loop instead
kmsq1 = pd.read_csv('data/spatialGrids/kmsq1.csv')
kmsq1 = gpd.GeoDataFrame(kmsq1, crs="EPSG:4326", geometry=kmsq1['geom'].apply(shapely.wkt.loads))
kmsq2 = pd.read_csv('data/spatialGrids/kmsq2.csv')
kmsq2 = gpd.GeoDataFrame(kmsq2, crs="EPSG:4326", geometry=kmsq2['geom'].apply(shapely.wkt.loads))
kmsq3 = pd.read_csv('data/spatialGrids/kmsq3.csv')
kmsq3 = gpd.GeoDataFrame(kmsq3, crs="EPSG:4326", geometry=kmsq3['geom'].apply(shapely.wkt.loads))
kmsq4 = pd.read_csv('data/spatialGrids/kmsq4.csv')
kmsq4 = gpd.GeoDataFrame(kmsq4, crs="EPSG:4326", geometry=kmsq4['geom'].apply(shapely.wkt.loads))
kmsq5 = pd.read_csv('data/spatialGrids/kmsq5.csv')
kmsq5 = gpd.GeoDataFrame(kmsq5, crs="EPSG:4326", geometry=kmsq5['geom'].apply(shapely.wkt.loads))
filterPSpoly = pd.read_csv('data/spatialGrids/filterPSpoly.csv')
filterPSpoly = gpd.GeoDataFrame(filterPSpoly, crs="EPSG:4326", geometry=filterPSpoly['geom'].apply(shapely.wkt.loads))

#1. get data
vmsOp2 = pd.read_csv('data/vmsOp2.csv')
#need to filter more than 2021?
vmsOp2 = gpd.GeoDataFrame(vmsOp2, geometry=vmsOp2['geom'].apply(shapely.wkt.loads), crs='EPSG:4326')
vmsOp2 = vmsOp2.drop('geom', axis='columns').rename(columns={'geometry':'geom'}).set_geometry('geom')

#2. model predictions
if TEST_THE_MODEL == 'multiclassModel' or TEST_THE_MODEL == 'binaryModel':
    vmsOp2_x = vmsOp2[['Speed', 'CourseCorrected', 'cogDiff', 'speedDiff+1', 'speedDiff-1', 'DayTime2', 'bufferCount']]
    vmsOp2_y_pred = MLmodel.best_estimator_.predict(vmsOp2_x)
    #merge model predictions to original dataset
    vmsOp2_pred = pd.concat([vmsOp2.reset_index(), pd.DataFrame(pd.Series(vmsOp2_y_pred), columns=['OperationPred']).replace({'OperationPred':labelsDict})], join='outer', axis=1)
    vmsOp2_pred = vmsOp2_pred.set_geometry('geom')
if TEST_THE_MODEL == 'speedFilter':
    vmsOp2_pred = vmsOp2
    vmsOp2_pred['OperationPred'] = vmsOp2_pred['Fishing'] #there is already a column where speed<6.5kn is considered a fishing point
    vmsOp2_pred = vmsOp2_pred.replace({'OperationPred':labelsDict1}).set_geometry('geom')


#3. fishing effort estimates
#3.1. Filter by fishing poly (will create a new column 'OperationPred2' will keep 'OperationPred' value only if point is inside fishing polygon, otherwise will be false)
vmsOp2_pred['OperationPred2'] = vmsOp2_pred.apply(lambda row: (False if filterPSpoly.geometry.contains(row['geom'])[0] == False else row['OperationPred']), axis=1)

#3.3. Renaming
#columns renaming
vmsOp2_pred = vmsOp2_pred.rename(columns={'Operation':'ObservedOp', 'OperationPred2':'PredictedOp'})
#rename observed operations labels for binary and speedFilter models so can be comparable to pre
if TEST_THE_MODEL == 'binaryModel' or TEST_THE_MODEL == 'speedFilter': 
    vmsOp2_pred = vmsOp2_pred.replace({'ObservedOp':{'A':'AN', 'N':'AN'}})

#save models prediction data
vmsOp2_pred.to_csv('results/'+ TEST_THE_MODEL + '/fishingEffortCheck/vmsOp2_pred.csv')

#3.3. Calculate mean point frequency
vmsOp2_pred.Date = pd.to_datetime(vmsOp2_pred.Date, utc=True)
vmsOp2_pred['timeDiff'] = vmsOp2_pred.groupby(['TrackCode'])['Date'].diff().dt.total_seconds()
meanFreqMinVMS = vmsOp2_pred['timeDiff'].mean()/60
meanFreqMinVMS

#3.4. Estimates by kmsq grids
kmsq = {'1km':kmsq1, '2km':kmsq2,'3km':kmsq3, '4km':kmsq4, '5km':kmsq5}
kmsqVMSobsdict = {}
kmsqVMSpreddict = {}

for kmsqKey in kmsq:
    #get kmsq dataframe corresponding to loop
    kmsqDF = gpd.GeoDataFrame(kmsq.get(kmsqKey))
    #join vms dataset with grid. Exclude False points
    vmsOpKmsq = vmsOp2_pred[vmsOp2_pred['PredictedOp'] != False].sjoin(kmsqDF, how='left')
    #count number of points by predicted operation and calculate operation time
    for pred in ['ObservedOp', 'PredictedOp']:
        vmsOpKmsqSum = vmsOpKmsq.groupby(by=['kmsqId',pred], as_index=False)['TrackCode'].count().reset_index().rename(columns={'TrackCode':str('VMS_pointCount_' + pred)})   
        vmsOpKmsqSum[str('VMSOpTime(min)_'+pred)] = vmsOpKmsqSum[str('VMS_pointCount_' + pred)]*meanFreqMinVMS
        #merge geom and export
        vmsOpKmsqSumGeom = vmsOpKmsqSum.merge(kmsqDF[['kmsqId', 'geom']], how='left', on='kmsqId')
        #vmsOpKmsqSumGeom.to_csv(pathExport + kmsqKey + '_vms_'+pred+'.csv')
        if pred == 'ObservedOp':
            kmsqVMSobsdict[kmsqKey] = vmsOpKmsqSumGeom.rename(columns={'ObservedOp':'Operation'})
            
        if pred == 'PredictedOp': 
            kmsqVMSpreddict[kmsqKey] = vmsOpKmsqSumGeom.rename(columns={'PredictedOp':'Operation'})

#export merged dataframe
km = ['1km', '2km', '3km', '4km', '5km']
for k in km:
    len(kmsqVMSpreddict[k])
    kmsqVMSpredObs = kmsqVMSpreddict[k].merge(kmsqVMSobsdict[k], how='outer', on=['kmsqId', 'geom', 'Operation']).fillna(0).to_csv('results/'+TEST_THE_MODEL+'/fishingEffortCheck/' + k + '_VMSobsVMSpred.csv')




Testing the model binaryModel


8.411289549376798

983

584

383

278

209